# EDA
- 결론: 한 영상 묶음은 하나의 라벨로 묶여있다.

In [1]:
import os
import glob
import pickle
import json
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd


In [2]:
base_path = '/shared_data/focussu/109.학습태도_및_성향_관찰_데이터/3.개방데이터/1.데이터/Training'

file_list = ['00_01', '00_02', '00_03', '00_04', '00_05',
                '10_01', '10_02', '10_03',]

pkl_files = []
meta_files = []
for file in file_list:
    pkl_files.extend(glob.glob(f'{base_path}/01.원천데이터/TS_{file}*_p/*_result.pkl'))
    meta_files.extend(glob.glob(f'{base_path}/02.라벨링데이터/TL_{file}/*.json'))
print(f'pkl_files: {len(pkl_files)}')


pkl_files: 333949


In [3]:
meta_list = []

for file in meta_files:
    if not os.path.exists(file):
        print(f"경고: 디렉토리가 존재하지 않습니다: {file}")
        continue
    with open(file, 'r') as f:
        metadata = json.load(f)
        meta_dict = {
                'filename': metadata['이미지']['filename'],
                'format': metadata['이미지']['format'],
                'idx': metadata['이미지']['timeline']['id'],
                'category_id': metadata['이미지']['category']['id'],
                'category_name': metadata['이미지']['category']['name'],
                'emotion': metadata['이미지']['emotion']
        }
        meta_list.append(meta_dict)
print(len(meta_list))

333949


In [4]:
df = pd.DataFrame(meta_list)
df.sort_values(by=['filename', 'idx'], inplace=True)

In [5]:
print(df.shape)

(333949, 6)


In [6]:
for k, v in df.groupby(by=['filename']):
    if(v['category_id'].nunique() > 1):
        

IndentationError: expected an indented block (1905653142.py, line 3)

In [10]:
df.head()

,filename,idx,category_id,category_name,emotion
0,04-01-378-01-1-8-23082700000337-01.mp4,38,1,집중,흥미로움
1,04-01-423-02-2-9-23083100000020-01.mp4,38,1,집중,흥미로움
2,04-01-497-01-2-8-23083000000036-01.mp4,11,1,집중,흥미로움
3,04-01-544-01-1-8-23090100000009-01.mp4,17,1,집중,흥미로움
4,04-01-179-02-2-9-23082700000331-01.mp4,24,1,집중,흥미로움


In [14]:
from tqdm import tqdm
NotExists = 0
Sleep = 0
NotandSleep = 0
for file in tqdm(meta_list):
    name = file['format'].split('.')[0]
    pkl_file = glob.glob(f'{base_path}/01.원천데이터/*/{name}_result.pkl')[0]
    if not os.path.exists(pkl_file):
        print(f"경고: 디렉토리가 존재하지 않습니다: {pkl_file}")
        continue
    with open(pkl_file, 'rb') as f:
        data = pickle.load(f)
    
    if not data.face_landmarks:
        NotExists += 1
    if file['category_id'] == 3:
        Sleep += 1
    if not data.face_landmarks and file['category_id'] == 3:
        NotandSleep += 1
print(f'NotExists: {NotExists}')
print(f'Sleep: {Sleep}')
print(f'NotandSleep: {NotandSleep}')



  0%|          | 0/333949 [00:00<?, ?it/s]

100%|██████████| 333949/333949 [05:26<00:00, 1021.83it/s]

NotExists: 2938
Sleep: 6079
NotandSleep: 126
